The `autograd` package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

## Tensor

`torch.Tensor` is the central class of the package. 

If we require gradient tracking
- Set its attribute `.requires_grad` as `True`, it starts to track all operations on it. 
- When you finish your computation you can call `.backward()` and have all the gradients computed automatically. 
- The gradient for this tensor will be accumulated into `.grad` attribute.

If we don't require gradient tracking
- To stop a tensor from tracking history, call `.detach()` to detach it from the computation history, and to prevent future computation from being tracked.
- To prevent tracking history (and using memory), we can also wrap the code block in with `torch.no_grad():`. 
(This can be particularly helpful when evaluating a model because the model may have trainable parameters with `requires_grad=True`, but for which we don’t need the gradients.
)




Another important class for autograd implementation is `Function`.

`Tensor` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation. 

- Each tensor has a `.grad_fn` attribute that references a `Function` that has created the `Tensor` (except for Tensors created by the user - their `grad_fn` is None).

- If we want to compute the derivatives, you can call `.backward()` on a Tensor. 

    - If Tensor is a scalar (i.e. it holds a one 
element data), we don’t need to specify any arguments to `backward()`
    - If it has more elements, we need to specify a gradient argument that is a tensor of matching shape.

In [2]:
import torch

Create a tensor and set `requires_grad=True` to track computation with it

In [3]:
x = torch.ones(2, 2, requires_grad=True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

Do a simple tensor operation:

In [4]:
y = x + 2
y # y was created as a result of an operation, so it has grad_fn

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

In [5]:
y.grad_fn

Do more operations on `y`:

In [6]:
z = y * y * 3
out = z.mean()

In [7]:
z

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)

In [8]:
out

tensor(27., grad_fn=<MeanBackward0>)

`.requires_grad_( ... )` changes an existing Tensor’s `requires_grad` flag **in-place**. The input flag defaults to `False` if not given.

In [9]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad) # should be False

# Now we set requires_grad to True explicitly
a.requires_grad_(True)
print(a.requires_grad)

b = a * a
print(b.grad_fn)

False
True


## Gradients

Consider a simple computation mentioned above: 

In [10]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()

Let's backprop now.

Because `out` contains a single scalar, `out.backward()` is equivalent to `out.backward(torch.tensor(1.))`

In [11]:
out.backward()

In [12]:
print(f"d(out)/dx : \n{x.grad}")

d(out)/dx : 
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


We should have got a matrix of ``4.5``. Let’s call the ``out``
*Tensor* “$o$”.
We have that $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.
Therefore,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.


Mathematically, if you have a vector valued function $\vec{y}=f(\vec{x})$,
then the gradient of $\vec{y}$ with respect to $\vec{x}$
is a Jacobian matrix:

\begin{align}J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\end{align}

Generally speaking, ``torch.autograd`` is an engine for computing
vector-Jacobian product. That is, given any vector
$v=\left(\begin{array}{cccc} v_{1} & v_{2} & \cdots & v_{m}\end{array}\right)^{T}$,
compute the product $v^{T}\cdot J$. If $v$ happens to be
the gradient of a scalar function $l=g\left(\vec{y}\right)$,
that is,
$v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$,
then by the chain rule, the vector-Jacobian product would be the
gradient of $l$ with respect to $\vec{x}$:

\begin{align}J^{T}\cdot v=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\left(\begin{array}{c}
   \frac{\partial l}{\partial y_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial y_{m}}
   \end{array}\right)=\left(\begin{array}{c}
   \frac{\partial l}{\partial x_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial x_{n}}
   \end{array}\right)\end{align}

(Note that $v^{T}\cdot J$ gives a row vector which can be
treated as a column vector by taking $J^{T}\cdot v$.)

This characteristic of vector-Jacobian product makes it very
convenient to feed external gradients into a model that has
non-scalar output.

Let's take a look at another example, in which the result is not a scalar

In [31]:
x = torch.ones(3, requires_grad=True)
def f(x):
  y = x * 2
  z = y * y * 2
  return z

print(f"f(x): {f(x)}") # z is a vector, not a scalar

f(x): tensor([8., 8., 8.], grad_fn=<MulBackward0>)


Now in this case `z` is no longer a scalar. `torch.autograd` could not compute the full Jacobian directly, but if we just want the vector-Jacobian product, simply pass the vector to backward as argument:

In [32]:
v = torch.tensor([1, 2, 3], dtype=torch.float)
f(x).backward(v, retain_graph=True)
# torch.autograd.backward(z, grad_tensors=v, retain_graph=True)

print(f"gradint of x: {x.grad}")

gradint of x: tensor([16., 32., 48.])


In [33]:
x.grad.zero_() # clear previous gradient otherwise it will get accumulated
unit_tensor = torch.ones(3, dtype=torch.float)
f(x).backward(unit_tensor, retain_graph=True)
print(f"gradint of x: {x.grad}")

gradint of x: tensor([16., 16., 16.])


In [ ]:
y.grad

You can also stop autograd from tracking history on Tensors
with ``.requires_grad=True`` either by wrapping the code block in
``with torch.no_grad():``

In [ ]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

Or by using ``.detach()`` to get a new Tensor with the same
content but that does not require gradients:



In [ ]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())